### 1. Environment variable setting block:

The following block is required to set environment variables that are read during the execution of the program code. 

User can change these environment variables between runs.

In [1]:
# Preliminary setup of experimental environment
import os
from pathlib import Path
import subprocess

nntile_dir = Path.cwd() / ".."

# Set environment variables
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # Limit CUDA visibility
os.environ["OMP_NUM_THREADS"] = "1" # Disable BLAS parallelism
os.environ["PYTHONPATH"] = str(nntile_dir / "build" / "wrappers" / "python") # Path to a binary dir of NNTile Python wrappers

# All StarPU environment variables are available at https://files.inria.fr/starpu/doc/html/ExecutionConfigurationThroughEnvironmentVariables.html
os.environ["STARPU_NCPU"] = "1" # Use only 1 CPU core
os.environ["STARPU_NCUDA"] = "1" # Use only 1 CUDA device
os.environ["STARPU_SILENT"] = "1" # Do not show lots of StarPU outputs
os.environ["STARPU_SCHED"] = "dmdasd" # Name StarPU scheduler to be used
os.environ["STARPU_FXT_TRACE"] = "0" # Do not generate FXT traces
os.environ["STARPU_WORKERS_NOBIND"] = "1" # Do not bind workers (it helps if several instances of StarPU run in parallel)
os.environ["STARPU_PROFILING"] = "1" # This enables logging performance of workers and bandwidth of memory nodes
os.environ["STARPU_BUS_STATS"] = "1" # This enables logging of bus usage, prined at the end of execution
os.environ["STARPU_HOME"] = str(Path.cwd() / "starpu") # Main directory in which StarPU stores its configuration files
os.environ["STARPU_PERF_MODEL_DIR"] = str(Path(os.environ["STARPU_HOME"]) / "sampling") # Main directory in which StarPU stores its performance model files
os.environ["STARPU_PERF_MODEL_HOMOGENEOUS_CPU"] = "1" # Assume all CPU cores are equal
os.environ["STARPU_PERF_MODEL_HOMOGENEOUS_CUDA"] = "1" # Assume all CUDA devices are equal
os.environ["STARPU_HOSTNAME"] = "GPT_NeoX_LMHead_example" # Force the hostname to be used when managing performance model files
os.environ["STARPU_FXT_PREFIX"] = str(Path(os.environ["STARPU_HOME"]) / "fxt") # Directory to store FXT traces if enabled

### 2. Data Preparation Block: 

This block uses the interpreted file "causal_lm_data_preparation.py". This Python script supports the following arguments when run:
- hf-dataset, (default=`"roneneldan/TinyStories"`): The name of the dataset to be processed and prepared for use in the training process. By default, the "TinyStories" dataset from the Huggingface infrastructure is specified,
- dataset-path, (default=`".data"`): path to the directory where previously downloaded datasets from remote sources are saved, making it easy to access for the future use,
- dataset-select, (`int`, default=`100`): specifies the number of records from the original dataset that fall into the training set,
- hf-tokenizer, (`str`, default=`"kimihailv/llama-1.3b"`): specifies the repository from the Huggingface infrastructure used as a tokenizer,
- tokenizer-path, (`str`, default=`".model"`): path to the directory where previously downloaded tokenizers are saved,
- seq-len, (`int`, default=`1024`): length of the input token sequence for the training process,
- batch-size, (`int`, default=`1`): batch size for the training process, then is the number of input data sentences between which the loss function optimizer step is called.

In [2]:
# Prepare TinyStories dataset into train.bin file
!python ../wrappers/python/examples/causal_lm_data_preparation.py \
    --hf-tokenizer="EleutherAI/gpt-neox-20b" --seq-len=256 --batch-size=256 --dataset-select=5000

### 3. Example Scenarios

Below we show an example of utilizing the GPT-NeoX model, implemented using the NNTile framework. We explore the following scenarios:

- **Training the model from a random initial state and saving it to a checkpoint.**
- **Loading the model weights from a checkpoint and continuing training with a different data type.**

For training and continuing retraining scenarios, the interpreted file "gpt_neox_training.py" is used. This Python script supports the following arguments when running:

- remote_model_name, (`str`, default=`"EleutherAI/gpt-neox-20b"`): This parameter specifies the name of the GPT-NeoX based model that resides within the HuggingFace infrastructure and will be utilized to initialize the configuration and the intial state of the NNTile model.

- pretrained, (choices=`["local", "remote"]`, default=`"local"`): This flag indicates the location of the pretrained model, with the `local` option requiring a configuration path (`config-path`) to start training from a randomly initialized state unless the checkpoint (`checkpoint-path`) is provided, in which case training continues from the last saved checkpoint state.

- checkpoint-path, (`str`, default=`""`): This refers to the file path where a saved checkpoint can be found, allowing for the resumption of training from a specific point if available.

- config-path, (`str`, default=`""`): This denotes the path to the configuration .json file that must be provided in the current version if the `pretrained` parameter is set to `"local"`.

- save-checkpoint-path, (`str`, default=`".model"`): This parameter specifies the directory path where intermediate checkpoints will be saved during the training process for future reference.

- optimizer, (choices=`["sgd", "adam", "adamw"]`, default=`"adam"`): This defines the type of optimizer that will be employed during the training process; the current version of NNTile supports three distinct optimization methods.

- model-path, (`str`, default=`".model"`): This indicates the directory path where previously loaded remote models are stored, facilitating easy access for further use.

- seq-len, (`int`, default=`1024`): length of the input token sequence for training.

- seq-len-tile, (`int`, default=`1024`): split size of sequence length into tiles

- batch-size, (`int`, default=`1`): batch size for the training process, which specifies the number of sentences processed by seq-len tokens between steps of the loss function optimizer.

- minibatch-size, (`int`, default=`-1`): batch size for which memory is allocated during training. The entire batch is divided into whole minibatches. All minibatches from one batch are fed through the model one by one to accumulate parameter gradients.

- minibatch-size-tile, (`int`, default=`-1`): batch size that goes to the CPU or GPU for calculations. Each minibatch must be divisible by an integer number of minibatch tiles.

- hidden-size-tile, (`int`, default=`-1`): the size of the pieces (tiles) into which the "hidden size" dimension is divided (also known as "embedding size") – the size of the multidimensional space into which incoming tokens are mapped. Only "piecewise" tensors of size hidden-size-tile along the corresponding axis are processed on the CPU and GPU.

- intermediate-size-tile, (`int`, default=`-1`): the size of the pieces (tiles) into which the "intermediate size" dimension is divided. Only "piecewise" tensors of size intermediate-size-tile along the corresponding axis are processed on the CPU and GPU.

- n-head-tile, (`int`, default=`-1`): the size of the pieces (tiles) into which the number of heads of the transformer layer is divided. Only “piecewise” tensors with a size of n-head-tile along the corresponding axis are processed by the CPU and GPU.

- dtype, (choices=`["fp32", "fp64", "tf32", "bf16", "fp32_fast_fp16", "fp32_fast_bf16"]`, default=`"fp32"`): This parameter outlines the various data types supported by NNTile, allowing users the flexibility to choose based on their model requirements.

- restrict, (choices=`["cpu", "cuda", None]`, default=`None`): This option allows users to specify restrictions on the computational resources utilized during training; selecting `"cpu"` restricts training to CPU-only cores, `"cuda"` limits it to GPU cores, while setting it to None allows for training across all available cores.

- use-redux, (action=`"store_true"`): a boolean flag that, when used in the argument string, allows for the computation of dependent tasks simultaneously, with the subsequent reduction of the results into a single tensor.

- dataset-path, (default=`".data"`): path to the directory where previously prepared datasets are saved.

- dataset-file, (default=`""`): path (relative to dataset-path) to the .bin file that is created in the block of data preparation for training.

- lr, (`float`, default=`1e-4`): step length for the optimization algorithm.

- nepochs, (`int`, default=`1`): number of full passes through the training set.

- logger, (action=`"store_true"`): a boolean flag that enables special logger thread, that forwards logs to a provided remote logger server

- logger-server-addr, (default=`"localhost"`): remote logger server URL

- logger-server-port, (`int`, default=`5001`): remote logger server port

#### 3.1. Training from the random initial state and saving into checkpoint.

This requires option `pretrained` set to `local` and `config-path` to point on previously created `.json` configuration file.

In this example, we start training in the fp32 type.

In [3]:
# Launch an external python process to train gpt_neox model on TinyStories
!python ../wrappers/python/examples/gpt_neox_training.py \
    --restrict="cuda" --pretrained=local --config-path="../wrappers/python/examples/gpt_neox_config.json" \
    --save-checkpoint-path=".model/nntile_checkpoint.pt" --optimizer="adam" --seq-len=256 --lr=1e-4 --dtype=fp32 --nepochs=1 \
    --batch-size=256 --minibatch-size=8 --dataset-file="tinystories/train.bin"

2025-05-20 16:56:33.154669: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-20 16:56:33.173347: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-20 16:56:33.178909: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-20 16:56:33.193580: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-20 16:56:34.426138: W tensorflow/compiler/tf2

#### 3.2. Resume training from the local checkpoint.

This requires option `pretrained` again to be set to `local`, `config-path` to point on previously created `.json` configuration file, and also `checkpoint-path` to point on the pre-existing checkpoint file in the PyTorch format.

Training process can be resumed using a different data type and on a different set of compute nodes. For example, here we switch to the TF32 type (synonymous with the fp32_fast_tf32 type) and restrict to using only GPUs.

In [3]:
# Launch an external python process to finetune a pretrained NNTile gpt_neox model on TinyStories
!python ../wrappers/python/examples/gpt_neox_training.py \
    --restrict="cuda" --pretrained=local --checkpoint-path=".model/nntile_checkpoint.pt" \
    --config-path="../wrappers/python/examples/gpt_neox_config.json" \
    --save-checkpoint-path=".model/nntile_further_checkpoint.pt" --optimizer="adam" --seq-len=256 --lr=1e-4 --dtype=tf32 \
    --restrict="cuda" --nepochs=1 --batch-size=256 --minibatch-size=8 --dataset-file="tinystories/train.bin"

2025-05-20 17:02:28.524686: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-20 17:02:28.549141: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-20 17:02:28.556447: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-20 17:02:28.572857: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-20 17:02:29.727950: W tensorflow/compiler/tf2

### 4. Inference process.

In the current version of the GPT-NeoX scenario, the NNTile framework model is created from a (pre-)loaded pre-trained GPT-NeoX model from the Huggingface library. The model layer weights are passed to the corresponding NNTile model layers, and then the inference process is performed solely by NNTile, without any involvement of third-party models and mechanisms. To perform the inference, we use another program file - "gpt_neox_generate.py". The program code supports the following arguments when running:

- cache_dir, (`str`, default="cache_hf"): path to the directory where previously loaded models from a remote source are saved,
- max-seq-len, (`int`, default=1024): maximum length of the input token sequence,
- remote-model-name, (`str`, default=`"EleutherAI/gpt-neox-20b"`): This parameter specifies the name of the GPT-NeoX based model that resides within the HuggingFace infrastructure and will be utilized to initialize the configuration and the intial state of the NNTile model.
- restrict, (choices=`["cpu", "cuda", None]`, default=`None`): limit on the computing resources used during inference; `"cpu"` restricts inference to CPU cores only, `"cuda"` - to GPU cores only, while None allows using all available cores,
- prompt, (`str`, default=`"What do you think about dogs?"`): input query, a string fed to the model input to perform inference based on it,
- generation-mode, (choices = `["Greedy", "TopK", "TopP"]`, default=`"Greedy"`): token generation mode in the GenerationMode class object (described in the "llm_params.py" file),
- parallel-sampling-mode, (choices=`["BeamSearch", "Parallel"]`, default=`"BeamSearch"`): parallel generation mode for multiple responses to a single query in the ParallelSamplingMode class object (described in the "llm_params.py" file),
- max-tokens, (`int`, default=`100`): maximum number of generated tokens, including user request tokens,
- use-cache, (action=`"store_true"`): boolean flag, when used in the argument line, enables the use of KV caches, allowing to reuse previously calculated values,
- top-k, (`int`, default=`None`): probabilistic selection based on the top-k most probable tokens,
- top-p-thr, (`float`, default=`None`): probabilistic selection based on tokens whose probability is not lower than the top-p-thr threshold,
- temperature, (`float`, default=`1.0`): "temperature" parameter for token generation,
- num-beams, (`int`, default=`1`): number of beams for parallel generation mode.

#### 4.1. Examples with different types of generation strategies

`BeamSearch` generation strategy and number of beams set to the default value of `1`.

In [3]:
!python ../wrappers/python/examples/gpt_neox_generate.py --cache_dir=.model --max-seq-len=512 \
    --restrict=cuda --use-cache \
    --prompt="Why does the Sun shine?" \
    --max-tokens=20

2025-07-03 18:50:41.851713: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-03 18:50:41.868071: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-03 18:50:41.873072: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-03 18:50:41.886100: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-03 18:50:43.001510: W tensorflow/compiler/tf2

`Parallel` generation strategy and number of beams set to `3`.

In [4]:
!python ../wrappers/python/examples/gpt_neox_generate.py --cache_dir=.model --max-seq-len=512 \
    --restrict=cuda --use-cache \
    --num-beams=3 --parallel-sampling-mode=Parallel \
    --prompt="Why does the Sun shine?" \
    --max-tokens=20

2025-07-03 18:51:41.393082: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-03 18:51:41.414817: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-03 18:51:41.421476: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-03 18:51:41.439128: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-03 18:51:42.619966: W tensorflow/compiler/tf2

`BeamSearch` generation strategy and number of beams set to `3`.

In [5]:
!python ../wrappers/python/examples/gpt_neox_generate.py --cache_dir=.model --max-seq-len=512 \
    --restrict=cuda --use-cache \
    --num-beams=3 --parallel-sampling-mode=BeamSearch \
    --prompt="Why does the Sun shine?" \
    --max-tokens=20

2025-07-03 18:52:53.509998: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-03 18:52:53.551888: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-03 18:52:53.564750: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-03 18:52:53.594908: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-03 18:52:54.799211: W tensorflow/compiler/tf2

#### 4.2. Examples with different token generation modes and temperatures

`TopK` token generation strategy with default temperature value of `1.0`.

In [6]:
!python ../wrappers/python/examples/gpt_neox_generate.py --cache_dir=.model --max-seq-len=512 \
    --restrict=cuda --use-cache \
    --generation-mode=TopK --top-k=10 \
    --prompt="Why does the Sun shine?" \
    --max-tokens=20

2025-07-03 18:54:04.710560: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-03 18:54:04.727317: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-03 18:54:04.732394: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-03 18:54:04.746267: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-03 18:54:05.870492: W tensorflow/compiler/tf2

`TopK` token generation strategy with the temperature value of `100.0`.

In [7]:
!python ../wrappers/python/examples/gpt_neox_generate.py --cache_dir=.model --max-seq-len=512 \
    --restrict=cuda --use-cache \
    --generation-mode=TopK --top-k=10 --temperature=100.0 \
    --prompt="Why does the Sun shine?" \
    --max-tokens=20

2025-07-03 18:55:15.799086: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-03 18:55:15.822637: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-03 18:55:15.829856: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-03 18:55:15.847507: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-03 18:55:17.064537: W tensorflow/compiler/tf2

`TopK` token generation strategy with the temperature value of `0.01`.

In [8]:
!python ../wrappers/python/examples/gpt_neox_generate.py --cache_dir=.model --max-seq-len=512 \
    --restrict=cuda --use-cache \
    --generation-mode=TopK --top-k=10 --temperature=0.01 \
    --prompt="Why does the Sun shine?" \
    --max-tokens=20

2025-07-03 18:56:25.810939: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-03 18:56:25.834406: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-03 18:56:25.841541: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-03 18:56:25.859554: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-03 18:56:26.968663: W tensorflow/compiler/tf2

`TopP` token generation strategy with default temperature value of `1.0`.

In [9]:
!python ../wrappers/python/examples/gpt_neox_generate.py --cache_dir=.model --max-seq-len=512 \
    --restrict=cuda --use-cache \
    --generation-mode=TopP --top-p=0.1 \
    --prompt="Why does the Sun shine?" \
    --max-tokens=20

2025-07-03 18:57:37.221875: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-03 18:57:37.245426: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-03 18:57:37.252392: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-03 18:57:37.269463: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-03 18:57:38.389408: W tensorflow/compiler/tf2

`TopP` token generation strategy with the temperature value of `100.0`.

In [12]:
!python ../wrappers/python/examples/gpt_neox_generate.py --cache_dir=.model --max-seq-len=512 \
    --restrict=cuda --use-cache \
    --generation-mode=TopP --top-p=0.1 --temperature=100.0 \
    --prompt="Why does the Sun shine?" \
    --max-tokens=20

2025-07-03 19:52:42.301524: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-03 19:52:42.319417: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-03 19:52:42.324680: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-03 19:52:42.338754: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-03 19:52:43.587996: W tensorflow/compiler/tf2

`TopP` token generation strategy with the temperature value of `0.01`.

In [11]:
!python ../wrappers/python/examples/gpt_neox_generate.py --cache_dir=.model --max-seq-len=512 \
    --restrict=cuda --use-cache \
    --generation-mode=TopP --top-p=0.1 --temperature=0.01 \
    --prompt="Why does the Sun shine?" \
    --max-tokens=20

2025-07-03 18:59:59.467025: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-03 18:59:59.489970: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-03 18:59:59.496930: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-03 18:59:59.514521: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-03 19:00:00.629828: W tensorflow/compiler/tf2